# Machine Learning project using simple regression 
Analysis to determine how certain factors impact the ZAR/USD exchange rate

In [40]:
#Import libraries
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import StandardScaler

In [7]:
#Create a pandas dataframe
rd_df = pd.read_csv('rand-dollar.csv', index_col = 0)

rd_df.head()

,ZAR/USD,Value of Exports (USD),Value of Exports (ZAR),Value of Imports (USD),Value of Imports (ZAR),IMF Reserve Position (USD),Foreign Exchange (USD),Claims on Non-residents (USD),Liabilities to Non-residents (USD),Savings Rate,Lending Rate,Government Bonds,"Financial Market Prices, Equities Index",Consumer Price Index
2008M01,7.01,5611.90,39356.82,8105.11,56841.94,1.92,29526.78,51547.61,37752.29,4.53,14.5,8.36,312.97,85.48
2008M02,7.66,6126.16,46946.33,8159.43,62527.78,2.01,29943.04,59702.31,45927.67,5.61,14.5,8.69,335.09,85.91
2008M03,7.97,6417.48,51150.89,8010.87,63851.12,2.05,30246.54,56448.18,42565.17,5.62,14.5,9.16,348.84,87.04
2008M04,7.79,7215.71,56174.34,9112.97,70944.49,2.03,30399.21,55534.75,43342.70,5.80,15.0,9.15,358.47,87.61
2008M05,7.62,7379.67,56240.45,9090.14,69275.97,2.10,30467.81,57682.89,46533.13,6.50,15.0,9.51,371.98,88.32


In [15]:
#Separate into features (X) and target (y)
X = rd_df[['Value of Exports (ZAR)']]
y = rd_df['ZAR/USD']

#Split into train and test sets (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

#Train a simple linear regression model
lm = LinearRegression()
lm.fit(X_train, y_train)

LinearRegression()

In [16]:
#Get the intercept
intercept_value = lm.intercept_
print(intercept_value)

3.2941036551619103


In [20]:
#Get the slope
slope_value = lm.coef_
print(slope_value)

[8.67712634e-05]


In [21]:
#Get the value the exchange rate in a month where the exports total are R100000
predicted_value = intercept_value + (slope_value * 100000)
print(predicted_value)

[11.97122999]


In [24]:
#Get the Mean Squared Error(MSE)

#Generate predictions for the test set
y_pred = lm.predict(X_test)

#Calculate MSE
mse = mean_squared_error(y_test, y_pred)
print(mse)

8.221852113297063


In [25]:
#Get the R² value
r_squared = lm.score(X_test, y_test)
print(r_squared)

-8.448868004702911


In [32]:
# ZAR/USD for August 2017
predicted_value_2017M08 = (intercept_value + (slope_value * 103199.17))[0]
print(predicted_value_2017M08)

12.24882601767889


In [34]:
#Get absolute error for the August 2017 prediction
actual_value = y_test[X_test['Value of Exports (ZAR)'] == 103199.17].values[0]
absolute_error = abs(actual_value - predicted_value_2017M08)
print(absolute_error)

0.9811739823211099


In [38]:
#Get correlations between predictors and the target variable
#Calculate the correlation matrix
correlation_matrix = rd_df.corr()

#Extract correlation between predictors and the target 
target_correlations = correlation_matrix['ZAR/USD'].abs()
print(target_correlations.sort_values(ascending = False))

ZAR/USD                                    1.000000
Consumer Price Index                       0.872660
IMF Reserve Position (USD)                 0.855311
Value of Imports (ZAR)                     0.791994
Value of Exports (ZAR)                     0.784756
Claims on Non-residents (USD)              0.727278
Liabilities to Non-residents (USD)         0.472230
Value of Imports (USD)                     0.350606
Value of Exports (USD)                     0.335780
Foreign Exchange (USD)                     0.317203
Government Bonds                           0.191455
Financial Market Prices, Equities Index    0.139858
Lending Rate                               0.056806
Savings Rate                               0.041555
Name: ZAR/USD, dtype: float64


In [47]:
#Prepare data
X = rd_df.drop(columns = ['ZAR/USD'])
y = rd_df['ZAR/USD']

#Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.2, shuffle=False)

#Train models
ridge = Ridge().fit(X_train, y_train)
lasso = Lasso(alpha = 0.01).fit(X_train, y_train)

In [48]:
#Calculate MSE for the ridge model
#Get predictions
y_train_pred_ridge = ridge.predict(X_train)

#Get the MSE
ridge_train_mse = mean_squared_error(y_train, y_train_pred_ridge)
print(ridge_train_mse)

0.04023298827614489


In [49]:
#Calculate MSE for the lasso model
#Get predictions
y_train_pred_lasso = lasso.predict(X_train)

#Get the MSE
lasso_train_mse = mean_squared_error(y_train, y_train_pred_lasso)

print(lasso_train_mse)

0.04695493921820657


In [50]:
#Calculate the testing MSE for the ridge model

#Generate predictions on the test data
y_test_pred_ridge = ridge.predict(X_test)

#Get MSE
ridge_test_mse = mean_squared_error(y_test, y_test_pred_ridge)

print(ridge_test_mse)

0.6323866944567396


In [51]:
#Calculate test MSE for the lasso model
#Get predictions
y_test_pred_lasso = lasso.predict(X_test)

#Get the MSE
lasso_test_mse = mean_squared_error(y_test, y_test_pred_lasso)

print(lasso_test_mse)

0.5790226574339904


In [53]:
#Determine which indicator is the best predictor of the target variable based on the Ridge Model Variable

#Get feature names and ridge coefficient
feature_names = rd_df.drop(columns = ['ZAR/USD']).columns
ridge_coefficient = ridge.coef_

#Create a DataFrame for easy readability
coef_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient (magnitude)': abs(ridge_coefficient)
}).sort_values('Coefficient (magnitude)', ascending = False)

print(coef_df)

                                    Feature  Coefficient (magnitude)
1                    Value of Exports (ZAR)                 1.061183
3                    Value of Imports (ZAR)                 1.002528
2                    Value of Imports (USD)                 0.736367
12                     Consumer Price Index                 0.630804
0                    Value of Exports (USD)                 0.578656
9                              Lending Rate                 0.525606
5                    Foreign Exchange (USD)                 0.155625
11  Financial Market Prices, Equities Index                 0.112937
8                              Savings Rate                 0.087518
6             Claims on Non-residents (USD)                 0.068125
4                IMF Reserve Position (USD)                 0.020487
10                         Government Bonds                 0.018772
7        Liabilities to Non-residents (USD)                 0.005712


In [54]:
#Determine which indicator is the best predictor of the target variable based on the Lasso Model Variable

#Get feature names and ridge coefficient
feature_names = rd_df.drop(columns = ['ZAR/USD']).columns
lasso_coefficient = lasso.coef_

#Create a DataFrame for easy readability
coef_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient (magnitude)': abs(lasso_coefficient)
}).sort_values('Coefficient (magnitude)', ascending = False)

print(coef_df)

                                    Feature  Coefficient (magnitude)
3                    Value of Imports (ZAR)                 1.171819
1                    Value of Exports (ZAR)                 1.135450
2                    Value of Imports (USD)                 0.816718
0                    Value of Exports (USD)                 0.577647
12                     Consumer Price Index                 0.239314
9                              Lending Rate                 0.209855
5                    Foreign Exchange (USD)                 0.080232
11  Financial Market Prices, Equities Index                 0.030944
7        Liabilities to Non-residents (USD)                 0.030399
4                IMF Reserve Position (USD)                 0.000000
6             Claims on Non-residents (USD)                 0.000000
8                              Savings Rate                 0.000000
10                         Government Bonds                 0.000000
